In [2]:
import cv2
import numpy as np
import os
from tqdm import tqdm
import json
import shutil
# show_image = True
show_image = False
from ddd_utils import draw_box_3d, project_to_image, compute_box_3d,rot_y2alpha,alpha2rot_y


In [6]:
data_path = "/home/fy/dataset/roadside/cvat/cvat114/"
dataname = "roadside"
bags_name = os.listdir(data_path+dataname)

In [9]:
bag_name = bags_name[0]
bag_dir = os.path.join(data_path,dataname, bag_name)
label_dir = "{}/label/".format(bag_dir)
label_files = os.listdir(label_dir)

In [10]:
label_files

['000186.json',
 '000144.json',
 '000092.json',
 '000243.json',
 '000111.json',
 '000123.json',
 '000315.json',
 '000242.json',
 '000191.json',
 '000011.json',
 '000172.json',
 '000160.json',
 '000262.json',
 '000030.json',
 '000150.json',
 '000138.json',
 '000028.json',
 '000126.json',
 '000164.json',
 '000275.json',
 '000304.json',
 '000036.json',
 '000230.json',
 '000121.json',
 '000017.json',
 '000095.json',
 '000257.json',
 '000261.json',
 '000290.json',
 '000059.json',
 '000042.json',
 '000231.json',
 '000280.json',
 '000256.json',
 '000277.json',
 '000053.json',
 '000068.json',
 '000298.json',
 '000307.json',
 '000238.json',
 '000026.json',
 '000161.json',
 '000099.json',
 '000190.json',
 '000107.json',
 '000286.json',
 '000122.json',
 '000018.json',
 '000247.json',
 '000054.json',
 '000041.json',
 '000225.json',
 '000239.json',
 '000209.json',
 '000322.json',
 '000051.json',
 '000183.json',
 '000321.json',
 '000148.json',
 '000301.json',
 '000196.json',
 '000115.json',
 '000320

In [4]:
def mkdir(path):
    if not os.path.exists(path):
        os.mkdir(path)


def loadlabel(label_path):
    with open(label_path) as f:
        labels = json.load(f)
        objs = []
        for label in labels:
            obj = {}
            position = label['psr']['position']
            rotation = label['psr']['rotation']['z']
            scale = label['psr']['scale']
            x, y, z = position["x"], position["y"], position["z"]
            xs, ys, zs = scale["x"], scale["y"], scale["z"]
            obj["obj_type"],obj["obj_id"]  = label['obj_type'],label['obj_id']
            obj["box"] = [x, y, z, xs, ys, zs, rotation]
            objs.append(obj)
    return objs


def save_label(labels_list,label_path):
    for labels in labels_list:
        path_split = label_path.split("/")
        file_name = "{}_c25/training/label/{}_{}_{}.txt".format(path_split[0],path_split[1],path_split[3].split(".")[0],labels[1])
        with open(file_name,"w") as f:
            if len(labels[0])==0:
                s = "others 0 0 0 0 0 0 0 0 0 0 0 0 0 0.0"
                f.write(s)
            for label in labels[0]:
                s = ""
                for l in label:
                    s+= str(l)
                    if not l is label[-1]:
                        s += " "
                if not label is labels[0][-1]:
                    s += "\n"
                f.write(s)


def save_calib(calibs,label_path):
    for cam in calibs:
        calib = calibs[cam]
        s = "P0:"
        for c in calib["intrinsic"]:
            s += " "
            s += str(c)
        path_split = label_path.split("/")
        file_name = "{}_c25/training/calib/{}_{}_{}.txt".format(path_split[0], path_split[1], path_split[3].split(".")[0],cam)
        with open(file_name, "w") as f:
            f.write(s)

def save_image(label_path):
    path_split = label_path.split("/")
    for cam in ["front","right","left"]:
        source_image_path = label_path.replace("/label","/camera/{}".format(cam)).replace(".json",".png")
        target_image_path = "{}_c25/training/images/trainval{}_{}_{}.png".format(path_split[0], path_split[1], path_split[3].split(".")[0],cam)
        shutil.copy(source_image_path,target_image_path)

def readcalib(bag_dir):
    calibs={}
    for cam in ["front","right","left"]:
        with open(os.path.join(bag_dir,"calib/camera/"+cam+".json")) as f:
            calib = json.load(f)
        # calib_ex = calib["extrinsic"]
        # calib_in = calib["intrinsic"]
        calib["intrinsic"].insert(4, 0.0)
        calib["intrinsic"].insert(8, 0.0)
        calib["intrinsic"].insert(12, 0.0)
        calibs[cam]= calib
    return calibs


def init_dir(dataname):
    mkdir("./{}_c25/".format(dataname))
    mkdir("./{}_c25/training/".format(dataname))
    mkdir("./{}_c25/training/images/".format(dataname))
    mkdir("./{}_c25/training/images/trainval".format(dataname))
    mkdir("./{}_c25/training/label/".format(dataname))
    mkdir("./{}_c25/training/calib/".format(dataname))

def get_imgs_path(label_path):
    imgs_path = {}
    for cam in ["front","right","left"]:
        img_path = label_path.replace("label","camera/{}".format(cam)).replace(".json",".png")
        imgs_path[cam] = img_path
    return imgs_path

# def cpimgs(dataname,bag_name,imgs_path):
#     for cam in ["front","right","left"]:
#         img_path =imgs_path[cam]
#         img_name = os.path.basename(img_path)
#         target_img_name = "{}_{}".format(bag_name,img_name).replace(".png","_{}.png".format(cam))
#         target_img_path = "./{}_c25/training/trainval/images/{}".format(dataname,target_img_name)
#         shutil.copy(img_path, target_img_path)

def compare_bbox(b2s,b3):
    min_loss = 1000
    limit_error = 300
    b3_c,b3_xy0,b3_xy1 = b3.mean(0),[max(b3[:, 0]),max(b3[:, 1])],[min(b3[:, 0]),min(b3[:, 1])]

    h = max(b3[:, 1]) - min(b3[:, 1])
    w = max(b3[:, 0]) - min(b3[:, 0])
    for b2 in b2s:
        b2c = np.array([b2[0][2]+b2[0][0],b2[0][3]+b2[0][1]])/2
        b2_xy0 = np.array([b2[0][0],b2[0][1]])
        b2_xy1 = np.array(([b2[0][2],b2[0][3]]))
        # b2_c = (b2_xy0+b2_xy1)/2
        # w,h = b2_xy1[0] - b2_xy0[0],b2_xy1[1] - b2_xy0[1]
        # w3,h3 = b3_xy1[0] - b3_xy0[0],b3_xy1[1] - b3_xy0[1]
        loss = (abs((b3_xy0-b2_xy0)).mean()+abs((b3_xy1-b2_xy1)).mean())+np.square(b3_c-b2c).mean()
        # print(loss,w,h)
        if loss <min_loss and loss<limit_error:
            min_loss = loss
            bbox_2d =[b2_xy0,b2_xy1]
    # print(min_loss)
    if min_loss<limit_error:
        return bbox_2d
    return [b3_xy0,b3_xy1]

class_map = {"Car":"Car","Pedestrian":"Pedestrian","Truck":"Truck",
             "Bus":"Bus","Motorcycle":"Motorcyclist","MotorcyleRider":"Motorcyclist",
             "Bicycle":"Cyclist","BicycleRider":"Cyclist","Van":"Van","Unknown":"Trafficcone"} #,"Van":"van",


def get_label(boxs_2d,boxs_3d):
    # bboxs_3d.append([dim, location, rotation_y, box_2d[-1],class_name])
    # col1 ： 类标签
    # col5-8 ：2D bbox，左上和右下坐标
    # col9-11 : 3D bbox dim 高、宽、长
    # col12-14 ：3D bbox centor 坐标
    # col15: 车体方向角
    # print(boxs_2d)
    labels = []
    for box_3d in boxs_3d:

        class_name = class_map[box_3d[4]]

        bbox_from3d = box_3d[3]
        [bbox_xy0,bbox_xy1] = compare_bbox(boxs_2d,bbox_from3d)
        dim = box_3d[0]
        location = [_[0] for _ in box_3d[1].tolist()]
        rotation = box_3d[2]
        label = [class_name,0,0,0,bbox_xy0[0],bbox_xy0[1],bbox_xy1[0],bbox_xy1[1]]+dim+location+[rotation]
        labels.append(label)

        # print(class_name)

    return labels



out_size = 400
world_size = 80
def project_3d_to_bird( pt):

    pt[0] += world_size / 2
    pt[1] = world_size - pt[1]
    pt = pt * out_size / world_size
    return pt.astype(np.int32)


def showddd(imgs_path,calibs,objs,yolodete):
    imgs = None
    bird_views = None
    labels_list = []
    for cam in ["left", "front", "right"]:
    # for cam in ["left"]:
        bboxs_3d = []
        img = cv2.imread(imgs_path[cam])
        bird_view = np.ones((out_size, out_size, 3), dtype=np.uint8) * 230
        # print(imgs_path[cam])
        bbox_2d = yolodete.dect(imgs_path[cam])
        # for bbox in bbox_2d:
        #     # print(bbox)
        #     if len(bbox)<1:
        #         continue
        #     xyxy = bbox[0]
        #     # print((xyxy[0],xyxy[1]))
        #     cv2.rectangle(img,(int(xyxy[0]),int(xyxy[1])),(int(xyxy[2]),int(xyxy[3])),(1,0,0),2)


        for obj in objs:
            [x, y, z, xs, ys, zs, r] = obj["box"]
            z -= zs/2
            rotation_y = -r
            if cam == "front":
                rotation_y -= 1.57
            dim = [zs, ys, xs]
            class_name = obj["obj_type"]

            location = [x, y, z]
            location = np.expand_dims(np.concatenate([location,np.ones(1)],0),1)
            p = np.array(calibs[cam]["extrinsic"]).reshape((4,4))[:3]
            location = np.dot(p, location)
            # print(class_name, location.T)

            calib = np.array(calibs[cam]["intrinsic"]).reshape((3,4))
            box_3d = compute_box_3d(dim, location, rotation_y)
            rect = box_3d[:4, [0, 2]]
            for k in range(4):
                rect[k] = project_3d_to_bird(rect[k])
            lc = (250, 152, 12)
            cv2.polylines(
                bird_view, [rect.reshape(-1, 1, 2).astype(np.int32)],
                True, lc, 2, lineType=cv2.LINE_AA)

            if box_3d.mean(0)[2] <0:
                continue
            box_2d = project_to_image(box_3d, calib)
            h = max(box_2d[:,1]) - min(box_2d[:,1])
            w = max(box_2d[:,0]) - min(box_2d[:,0])
            if abs(h) <10 or abs(w)<10  :
                continue
                # print("wh",class_name,box_2d[0][0]-box_2d[7][0],box_2d[0][1]-box_2d[7][1])
            if min(box_2d[-1])<-100 or box_2d[-1][0]>1400 or box_2d[-1][1]>900 :
                continue
            img = draw_box_3d(img, box_2d)

            bboxs_3d.append([dim, location, rotation_y, box_2d,class_name])

        labels = get_label(bbox_2d, bboxs_3d)

        for l in labels:
            bbox =l[4:8]
            xyxy = bbox
            # print((xyxy[0],xyxy[1]))
            cv2.rectangle(img, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (255, 255, 0), 2)

        labels_list.append([labels,cam])
        if show_image:
            if cam=="left":
                imgs = img
            else:
                imgs = np.hstack((imgs,img))

            if cam=="left":
                bird_views = bird_view
            else:
                bird_views = np.hstack((bird_views,bird_view))
    if show_image:
        imgs = cv2.resize(imgs,(imgs.shape[1]//2,imgs.shape[0]//2))
        cv2.imshow("f",imgs)
        cv2.imshow("f2",bird_views)
        cv2.waitKey(1)
    return labels_list
